# Executive Order Sentiment & Bias

Evaluate executive orders for sentiment and bias using technique outlined in https://arxiv.org/pdf/2312.06281.

In [1]:
import json
import os
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas
from pandas import DataFrame
import dateparser

In [ ]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_retries=3
)

# extract some debugging stats from LLM results
def build_stats(result):
    return {
        'input_tokens': result['raw'].usage_metadata['input_tokens'],
        'output_tokens': result['raw'].usage_metadata['output_tokens']
    }

# eos to process, we used the masked ones to avoid bias from presidential names
source_dir = 'data/executive_orders/masked/'

# process all executive orders with a given pipeline
def process_eos(
    pipeline: any,
    target_dir: str,
    log_path: str,
    *,
    concurrency = 8,
    first_n = None
):
    def process_eo(eo_path):
        log = { 'path': eo_path }

        start_time = time.time()
        try:
            # process the contents of EO
            with open(source_dir + eo_path) as source_file:
                eo = json.load(source_file)
                result = pipeline.invoke({'text': '\n'.join(eo["content"])})
                log = log | build_stats(result)

            # write the result to the target directory
            with open(target_dir + eo_path, 'w') as target_file:
                json.dump(result['parsed'].model_dump(), target_file, indent=4)
        except Exception as e:
            print(f'Error processing {eo_path}: {str(e)}')
            log['error'] = str(e)

        duration = time.time() - start_time
        log['duration'] = duration

        return log

    # figure out which docs to process
    eo_paths = os.listdir(source_dir)
    eo_paths.sort()
    if first_n:
        eo_paths = eo_paths[:first_n]

    # use a threadpool to process the eos concurrently
    results = []
    with ThreadPoolExecutor(max_workers=concurrency) as executor:
        futures = [executor.submit(process_eo, path) for path in eo_paths]

        for future in as_completed(futures):
            results.append(future.result())

    df = DataFrame(results)
    df.to_csv(log_path, index=False)
    return df

## Sentiment
Analyze for 5 primary emotions.

In [27]:
class EmotionScores(BaseModel):
    joy: int = Field(description="Joy conveyed on a scale of 1-10.")
    sadness: int = Field(description="sadness conveyed on a scale of 1-10.")
    fear: int = Field(description="fear conveyed on a scale of 1-10.")
    disgust: int = Field(description="disgust conveyed on a scale of 1-10.")
    surprise: int = Field(description="surprise conveyed on a scale of 1-10.")

class EmotionResponse(BaseModel):
    initial_scores: EmotionScores = Field(description="Initial emotion scores.")
    critique: str = Field(description="Critique of the initial scores.")
    revised_scores: EmotionScores = Field(description="Revised emotion scores.")

sentiment_pipeline = ChatPromptTemplate.from_template(
    '\n '.join([
        'Your task is to understand and score the sentiment the President is conveying in the following document by considering the content, context, and language.',
        '- Is the President attempting to portray joy, sadness, anger, fear, disgust, or surprise with the document?',
        '- For each emotion, score the relative intensity on a scale of 1-10 that the American public is expected to feel.',
        '- Do not consider text that are not part of the document and use a score of 1 if not enough information is provided.',
        '- Critique your answer by thinking through it step by step.',
        '- Finally, give your revised scores.',
        '# Document',
        '{text}',
    ])
) | model.with_structured_output(EmotionResponse, include_raw=True)

In [ ]:
process_eos(
    sentiment_pipeline,
    'data/executive_orders/sentiment/',
    'data/executive_orders/sentiment_log.csv'
)

## Political Bias
Analyze across 7 political axes.

In [35]:
class PoliticalBias(BaseModel):
    ideological_tilt: int = Field(description="-5 = Highly Conservative, 0 = Centrist/Moderate, +5 = Highly Liberal")
    immigration: int = Field(description="-5 = Strictly Restrictive (Closed), 0 = Balanced/Selective, +5 = Extremely Open")
    executive_power: int = Field(description="-5 = Strongly Restrains Executive Authority, 0 = Neutral/Balanced, +5 = Strongly Expands Executive Authority")
    federalism: int = Field(description="-5 = Heavy State Control, 0 = Cooperative Balance, +5 = Strong Federal Control")
    security_liberty: int = Field(description="-5 = Strong Liberties/Privacy Focus, 0 = Balanced Approach, +5 = Heavy National Security Focus")
    economic_regulation: int = Field(description="-5 = Strong Deregulation/Laissez-Faire, 0 = Moderate Regulation, +5 = Heavy Government Intervention")
    social_inclusion: int = Field(description="-5 = Emphasis on Conformity/Traditional Uniformity, 0 = Neutral, +5 = Emphasis on Diversity/Pluralism")

class PoliticalBiasResponse(BaseModel):
    initial_scores: PoliticalBias = Field(description="Initial political bias scores.")
    critique: str = Field(description="Critique of the initial scores.")
    revised_scores: PoliticalBias = Field(description="Revised political bias scores.")

political_bias_pipeline = ChatPromptTemplate.from_template(
    '\n '.join([
        'Your task is to consider each document and analyze its content, context, and language for political bias and idealogy along the following axes.',
        '- For each axis, score the document on a scale of -5 to +5 given the description of each axis.',
        '- Critique your answer by thinking through it step by step.',
        '- The mere fact of being an executive order should not indicate expansion of executive authority on its own.',
        '- Finally, give your revised scores.',
        '# Axes',
        '1. Ideological Tilt (Liberal vs. Conservative)', 
        '- Gauge where the document stands on the traditional left-right political spectrum (e.g., social welfare spending, cultural values, role of government).',
        '- -5 = Highly Conservative, 0 = Centrist/Moderate, +5 = Highly Liberal',
        '2. Immigration Approach (Closed vs. Open Borders)',
        '- Reflects how permissive or restrictive the document is regarding immigration, visas, refugees, and border enforcement.',
        '- -5 = Strictly Restrictive (Closed), 0 = Balanced/Selective, +5 = Extremely Open',
        '3. Executive Power (Restriction vs. Expansion)',
        '- Measures whether the document curtails presidential power (by imposing checks or limiting discretion) or broadens the executive\'s reach (allowing unilateral actions or emergency powers).',
        '- -5 = Strongly Restrains Executive Authority, 0 = Neutral/Balanced, +5 = Strongly Expands Executive Authority',
        '4. Federalism (State vs. Federal Primacy)',
        '- Evaluates if the documents shifts power and decision-making downward to states or upward to the federal government. A high positive score suggests centralizing authority in Washington, while a negative score favors decentralized/state-led governance.',
        '- -5 = Heavy State Control, 0 = Cooperative Balance, +5 = Strong Federal Control',
        '5. National Security vs. Civil Liberties',
        '- Captures whether the document prioritizes national security and law enforcement powers (potentially at civil liberties\' expense) or emphasizes individual rights and freedoms (possibly limiting security measures).',
        '- -5 = Strong Liberties/Privacy Focus, 0 = Balanced Approach, +5 = Heavy National Security Focus',
        '6. Economic Regulation (Deregulation vs. Intervention)',
        '- Determines how deeply the document involves the government in economic matters—whether it rolls back rules and fosters free markets or imposes policies (e.g., wage/price controls, mandatory guidelines) to steer the economy.',
        '- -5 = Strong Deregulation/Laissez-Faire, 0 = Moderate Regulation, +5 = Heavy Government Intervention',
        '7. Social Inclusion (Conformity vs. Diversity)',
        '- Assesses whether the document promotes inclusivity (racial, ethnic, religious, gender, LGBTQ+ protections) and encourages pluralism, or whether it trends toward uniform standards, fewer cultural accommodations, and more traditional/conforming policies.',
        '- -5 = Emphasis on Conformity/Traditional Uniformity, 0 = Neutral, +5 = Emphasis on Diversity/Pluralism',
        '# Document',
        '{text}'
    ])
) | model.with_structured_output(PoliticalBiasResponse, include_raw=True)

In [36]:
process_eos(
    political_bias_pipeline,
    'data/executive_orders/political_bias/',
    'data/executive_orders/political_bias_log.csv'
)

Error processing executive-order-12928-promoting-procurement-with-small-businesses-owned-and-controlled.json: 'NoneType' object has no attribute 'model_dump'
Error processing executive-order-14170-reforming-the-federal-hiring-process-and-restoring-merit-government.json: 'NoneType' object has no attribute 'model_dump'
Error processing executive-order-8052-designating-the-honorable-angel-r-de-jesus-acting-judge-the-district.json: 'NoneType' object has no attribute 'model_dump'


,path,input_tokens,output_tokens,duration,error
0,executive-order-0.json,1077,107,1.378071,NaN
1,executive-order-1.json,1077,87,1.466198,NaN
2,executive-order-10000-regulations-governing-ad...,5588,197,1.961811,NaN
3,commission-appointing-commissioners-for-survey...,1332,266,2.119709,NaN
4,executive-order-1000-alamo-national-forest-new...,1946,239,2.560071,NaN
...,...,...,...,...,...
10434,order-constituting-the-army-virginia.json,1319,116,1.282690,NaN
10435,executive-order-withdrawal-land-for-suppai-hav...,1250,490,3.285638,NaN
10436,executive-order-yuma-reserve.json,1238,402,2.798234,NaN
10437,order-general-hw-halleck.json,1153,218,1.882841,NaN


## Dataset

Combine all the results into a single dataset.

In [37]:
# load up the raw files to seed the dataset
raw_dir = 'data/executive_orders/raw/'
eos_raw = eo_paths = os.listdir(raw_dir)

dataset_rows = {}
for eo_path in eos_raw:
    with open(raw_dir + eo_path) as source_file:
        eo = json.load(source_file)
        dataset_rows[eo_path] = {
            'name': eo_path.replace('.json', ''),
            'president': eo['president'],
            'president_byline': eo['president_byline'],
            'date': eo['date'],
            'title': eo['title']
        }

# add the sentiment data
sentiment_dir = 'data/executive_orders/sentiment/'
sentiment_files = os.listdir(sentiment_dir)

for sentiment_file in sentiment_files:
    with open(sentiment_dir + sentiment_file) as f:
        try:
            eo = json.load(f)
            dataset_rows[sentiment_file] = dataset_rows[sentiment_file] | {
                'initial_joy': eo['initial_scores']['joy'],
                'initial_sadness': eo['initial_scores']['sadness'],
                'initial_fear': eo['initial_scores']['fear'],
                'initial_disgust': eo['initial_scores']['disgust'],
                'initial_surprise': eo['initial_scores']['surprise'],
                
                'revised_joy': eo['revised_scores']['joy'],
                'revised_sadness': eo['revised_scores']['sadness'],
                'revised_fear': eo['revised_scores']['fear'],
                'revised_disgust': eo['revised_scores']['disgust'],
                'revised_surprise': eo['revised_scores']['surprise']
            }
        except Exception as e:
            print('sentiment', sentiment_file, e)

# add the bias
political_bias_dir = 'data/executive_orders/political_bias/'
political_bias_files = os.listdir(political_bias_dir)

for political_bias_file in political_bias_files:
    with open(political_bias_dir + political_bias_file) as f:
        try:
            eo = json.load(f)
            dataset_rows[political_bias_file] = dataset_rows[political_bias_file] | {
                'initial_idelogical_tilt': eo['initial_scores']['ideological_tilt'],
                'initial_immigration': eo['initial_scores']['immigration'],
                'initial_executive_power': eo['initial_scores']['executive_power'],
                'initial_federalism': eo['initial_scores']['federalism'],
                'initial_security_liberty': eo['initial_scores']['security_liberty'],
                'initial_economic_regulation': eo['initial_scores']['economic_regulation'],
                'initial_social_inclusion': eo['initial_scores']['social_inclusion'],
                
                'revised_idelogical_tilt': eo['revised_scores']['ideological_tilt'],
                'revised_immigration': eo['revised_scores']['immigration'],
                'revised_executive_power': eo['revised_scores']['executive_power'],
                'revised_federalism': eo['revised_scores']['federalism'],
                'revised_security_liberty': eo['revised_scores']['security_liberty'],
                'revised_economic_regulation': eo['revised_scores']['economic_regulation'],
                'revised_social_inclusion': eo['revised_scores']['social_inclusion']
            }
        except Exception as e:
            print('bias', political_bias_file, e)

combined_df = pandas.DataFrame(dataset_rows.values())

# convert the date in the form `September 10, 1970` to `YYYY-MM-DD`
combined_df['date'] = combined_df['date'].apply(lambda x: dateparser.parse(x).strftime('%Y-%m-%d'))

combined_df

sentiment executive-order-the-civil-service-order.json Expecting value: line 1 column 1 (char 0)
bias executive-order-12928-promoting-procurement-with-small-businesses-owned-and-controlled.json Expecting value: line 1 column 1 (char 0)
bias executive-order-14170-reforming-the-federal-hiring-process-and-restoring-merit-government.json Expecting value: line 1 column 1 (char 0)
bias executive-order-8052-designating-the-honorable-angel-r-de-jesus-acting-judge-the-district.json Expecting value: line 1 column 1 (char 0)


,name,president,president_byline,date,title,initial_joy,initial_sadness,initial_fear,initial_disgust,initial_surprise,...,initial_security_liberty,initial_economic_regulation,initial_social_inclusion,revised_idelogical_tilt,revised_immigration,revised_executive_power,revised_federalism,revised_security_liberty,revised_economic_regulation,revised_social_inclusion
0,executive-order-12801-barring-overflight-takeo...,George Bush,41st President of the United States: 1989 ‐ 1993,1992-04-15,"Executive Order 12801-Barring Overflight, Take...",1.0,2.0,3.0,3.0,1.0,...,3.0,0.0,0.0,0.0,-1.0,1.0,0.0,4.0,0.0,0.0
1,executive-order-8452-exemption-albert-clyde-bu...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1940-06-20,Executive Order 8452-Exemption of Albert Clyde...,2.0,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,executive-order-9084-amending-executive-order-...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1942-03-03,Executive Order 9084-Amending Executive Order ...,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,executive-order-8510-establishing-the-carolina...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1940-08-08,Executive Order 8510-Establishing the Carolina...,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,executive-order-7776-modification-executive-or...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1937-12-27,Executive Order 7776-Modification of Executive...,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10434,executive-order-7935-exemption-j-humbird-smith...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1938-07-16,Executive Order 7935-Exemption of J. Humbird S...,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10435,executive-order-9492-regulations-governing-non...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1944-10-24,Executive Order 9492-Regulations Governing Non...,3.0,1.0,1.0,1.0,2.0,...,1.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,2.0,0.0
10436,executive-order-8377-amending-executive-order-...,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1940-03-18,Executive Order 8377-Amending Executive Order ...,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10437,executive-order-11557-enlarging-the-membership...,Richard Nixon,37th President of the United States: 1969 ‐ 1974,1970-09-10,Executive Order 11557-Enlarging the Membership...,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
combined_df.to_csv('data/executive_orders/sentiment.csv', index=False)